In [13]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [14]:
from evaluator.tester import Tester, train_ds, val_ds, test_ds

In [18]:
import re
from common.constants import (
    OPENROUTER_BASE_URL,
    OPENROUTER_API_KEY,
)
from models.item import Item
from openai import OpenAI

SYSTEM_PROMPT = """
You are a price estimator who estimates the price of a product based on its description.
Respond with only the price number in dollars. Do not include currency symbol, just respond with the number.
"""
MODEL = "deepseek/deepseek-r1-0528:free"


def get_messages(item: Item) -> list:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": "Estimate the price of this product" + "\n" + item.summary,
        },
    ]
    return messages


def llm_model(item: Item) -> float:
    openai_client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=OPENROUTER_API_KEY)
    response = openai_client.chat.completions.create(
        model=MODEL,
        messages=get_messages(item),
    )
    estimated_price = response.choices[0].message.content
    estimated_price = re.sub(r"[^0-9.]", "", estimated_price)
    try:
        estimated_price = float(estimated_price)
    except Exception as err:
        print(f"Error: {err}")
        return 0
    return estimated_price

In [19]:
Tester.test(llm_model)

1: Guess: $55.00 Truth: $35.00 Error: $20.00 SLE: 0.20 Item: Small Soar Dove Bird Necklace, Delicate ...
2: Guess: $25.00 Truth: $25.00 Error: $0.00 SLE: 0.00 Item: Omaha Steaks Gift Card
3: Guess: $350.00 Truth: $285.00 Error: $65.00 SLE: 0.04 Item: Ohana Baritone Ukulele Model BK-20CE
4: Guess: $34.99 Truth: $4.88 Error: $30.11 SLE: 3.28 Item: BodyGuardz - Pure 2 Edge Glass Screen Pr...
Error: could not convert string to float: '6.996.99'
5: Guess: $0.00 Truth: $5.00 Error: $5.00 SLE: 3.21 Item: Numer 1 Dad Iron On Decal, Father's Day ...
6: Guess: $24.99 Truth: $16.99 Error: $8.00 SLE: 0.14 Item: I Love My Maltese Rhinestone Iron on Tra...
7: Guess: $75.00 Truth: $84.99 Error: $9.99 SLE: 0.02 Item: JJ Electronics Amplifier Tubes (T-EL84-J...
8: Guess: $75.00 Truth: $18.99 Error: $56.01 SLE: 1.78 Item: Women’s Satin Kimono Robe for Bridesmaid...
9: Guess: $24.99 Truth: $22.08 Error: $2.91 SLE: 0.01 Item: Monoprice PLA Plus+ Premium 3D Filament ...
10: Guess: $55.00 Truth: $39.98 Erro

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1770768000000'}, 'provider_name': None}}, 'user_id': 'user_36bg20UCCuL3GwePUZWNoffdVw9'}